In [3]:
!pip install sagemaker -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 76.9 MB/s eta 0:00:00
  Attempting uninstall: platformdirs
    Found existing installation: platformdirs 3.11.0
    Uninstalling platformdirs-3.11.0:
      Successfully uninstalled platformdirs-3.11.0
  Attempting uninstall: mock
    Found existing installation: mock 5.1.0
    Uninstalling mock-5.1.0:
      Successfully uninstalled mock-5.1.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.227.0
    Uninstalling sagemaker-2.227.0:
      Successfully uninstalled sagemaker-2.227.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.2.2 which is incompatible.
virtualenv 20.21.0 requires platformdirs<4,>=2.4, but you have platformdirs 4.3.6 which is incompatible.


In [4]:
import sagemaker
import boto3

In [5]:
sess = sagemaker.Session()

sagemaker_session_bucket = None

if sagemaker_session_bucket is None and sess is not None:
    sagemaker_session_bucket = sess.default_bucket

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.Client("iam")
    role = iam.get_role(RoleName="sagemake_execution_role")["Role"]["Arn"]

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
print(f"sagemaker role arn: {role}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::187441937046:role/service-role/AmazonSageMaker-ExecutionRole-20240928T162976
sagemaker session region: us-east-1


In [6]:
from sagemaker.huggingface.model import HuggingFaceModel

In [9]:
hub = {
    "HF_MODEL_ID": 'distilbert-base-uncased-distilled-squad',
    "HF_TASK": 'question-answering'
}

hugging_face_model = HuggingFaceModel(
    env=hub,
    role=role,
    transformers_version="4.26",
    pytorch_version="1.13",
    py_version='py39',
)

predictor = hugging_face_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.2xlarge"
)

data = {"inputs": {
    "question": "What is used for inference?",
    "context": """My Name is Philipp and I live in Nuremberg. 
    This model is used with sagemaker for inference."""
    }
}

------!

In [10]:
predictor.predict(data)

{'score': 0.9987204670906067, 'start': 73, 'end': 82, 'answer': 'sagemaker'}

In [12]:
data = {
"inputs": {
    "question": "What does jaswanth do?",
    "context": """My Name is Jaswanth and
    I am working with ozonetel as speech and AI Intern."""
    }
}

In [13]:
predictor.predict(data)

{'score': 0.6187998056411743,
 'start': 33,
 'end': 78,
 'answer': 'working with ozonetel as speech and AI Intern'}